In [1]:
### Author: Leonie Küchenhoff
### Date: October 2022
### Purpose of script: Test stat. significance on fractions of SNP type per treatment condition
### on RNA and WGS data

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.special import expit
from itertools import product
from config import outdir, basedir, wgs_vartype_dir


In [ ]:
plt.rcParams.update({'font.size':18, 'text.usetex': False,
"svg.fonttype": 'none'})

In [3]:
names = ['279', '282', '450']
paths_anno = [f'{wgs_vartype_dir}/{i}_absolute_numbers.txt' for i in names]

In [4]:
abs_dict_snv = {}
for i in names:
    path = f'{wgs_vartype_dir}/{i}_absolute_numbers.txt'
    table = pd.read_csv(path, sep='\t')
    table['replicate'] = i
    abs_dict_snv[i] = table

### WGS data

In [5]:
abs_count = pd.concat((abs_dict_snv['279'], abs_dict_snv['282'], abs_dict_snv['450'])).fillna(0)
full_df = pd.melt(abs_count, id_vars = ['replicate','replacement'], value_vars = ['heart', 'tail'])
rest = full_df[full_df['replacement']!='TC'].groupby(['replicate','variable']).sum().reset_index()
rest['replacement'] = 'other'
df_AG = full_df[full_df['replacement']=='TC']
bin_data = pd.concat([df_AG, rest])
bin_data_large = pd.merge(rest,df_AG, on = ['replicate', 'variable'], how = 'outer', suffixes=['nonAG','AG'])
smf.glm("valuenonAG + valueAG ~ variable ", family=sm.families.Binomial(), data=bin_data_large).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -25.487
Date:                       Mon, 16 Jan 2023   Deviance:                       25.729
Time:                               17:04:03   Pearson chi2:                     22.7
No. Iterations:                            4   Pseudo R-squ. (CS):            0.09318
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.2826      0.197      6.519      0.000       0.897       1.668
variable[T.tail]    -0.1659      0.219     -0.759      0.448      -0.594       0.263
====================================================================================
"""

In [6]:
abs_count = pd.concat((abs_dict_snv['279'], abs_dict_snv['282'], abs_dict_snv['450'])).fillna(0)
full_df = pd.melt(abs_count, id_vars = ['replicate','replacement'], value_vars = ['liver', 'tail'])
rest = full_df[full_df['replacement']!='TC'].groupby(['replicate','variable']).sum().reset_index()
rest['replacement'] = 'other'
df_AG = full_df[full_df['replacement']=='TC']
bin_data = pd.concat([df_AG, rest])
bin_data_large = pd.merge(rest,df_AG, on = ['replicate', 'variable'], how = 'outer', suffixes=['nonAG','AG'])
smf.glm("valuenonAG + valueAG ~ variable ", family=sm.families.Binomial(), data=bin_data_large).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -23.780
Date:                       Mon, 16 Jan 2023   Deviance:                       21.866
Time:                               16:10:18   Pearson chi2:                     19.5
No. Iterations:                            4   Pseudo R-squ. (CS):             0.4734
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5315      0.195      7.856      0.000       1.149       1.914
variable[T.tail]    -0.4148      0.217     -1.911      0.056      -0.840       0.011
====================================================================================
"""

### RNA data

In [23]:
# In names list, two samples were swiched (032 was actually treated with nrch, 030 with pbs)
names = ['028_pbs_R', '029_pbs_R', '032_pbs_R','033_nrch_R', '030_nrch_R', '036_nrch_R', 
            '011_pbs', '012_nrch', '013_nrch', '014_nrch', '279_spry', '321_pbs', '333_pbs', '450_spry', '283_spry']
# treatment in same order as sample names
base_editor = ['pbs', 'pbs', 'pbs', '8e-NRCH', '8e-NRCH', '8e-NRCH',
            'pbs', '8e-NRCH', '8e-NRCH','8e-NRCH', 'max-SpRY', 'pbs', 'pbs', 'max-SpRY', 'max-SpRY']
varcallers = ['hc', 'pl', 'st']
mutation = ['r636q' if i[-1] == 'R' else 'p635l' for i in names]
combinations = list(product(names, varcallers))
pairing = dict(zip(names, base_editor))
pairing_mut = dict(zip(names, mutation))

In [24]:
strand=pd.read_csv(f"{outdir}/SNP_type/strand_tier.txt",sep=",")
no_strand=pd.read_csv(f"{outdir}/SNP_type/tier.txt",sep=",")

group_strand=pd.read_csv(f"{outdir}/SNP_type/strand.txt",sep=",")
group_no_strand=pd.read_csv(f"{outdir}/SNP_type/grouped.txt",sep=",")

In [25]:
def edit_df(df, col):
    df_c = df.copy()
    full_df = df_c.melt(id_vars = 'replacement')

    rest = full_df[full_df['replacement']!=col].groupby(['variable']).sum().reset_index()
    rest['replacement'] = 'other'

    df_AG = full_df[full_df['replacement']==col].groupby(['variable']).sum().reset_index()
    df_AG['replacement'] = 'AG'

    bin_data_large = pd.merge(rest,df_AG, on = ['variable'], how = 'outer', suffixes=['nonAG','AG'])
    bin_data_large['treatment'] = bin_data_large['variable'].map(pairing)
    bin_data_large['mutation'] = bin_data_large['variable'].map(pairing_mut)

    return(bin_data_large)

In [26]:
# stranded but no tiers

In [27]:
bin_data_large = edit_df(group_strand, col = 'AG')

In [28]:
# 8e-NRCH vs PBS r636q
data=bin_data_large[bin_data_large['mutation'] == 'r636q']
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -19.183
Date:                       Mon, 16 Jan 2023   Deviance:                       2.4916
Time:                               17:07:13   Pearson chi2:                     2.53
No. Iterations:                            5   Pseudo R-squ. (CS):             0.5428
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.4216      0.064     22.320      0.000       1.297       1.546
treatment[T.pbs]     0.2117      0.098      2.159      0.031       0.020       0.404
====================================================================================
"""

In [29]:
# max-SpRY vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != '8e-NRCH')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -18.073
Date:                       Mon, 16 Jan 2023   Deviance:                       4.2028
Time:                               17:07:20   Pearson chi2:                     4.07
No. Iterations:                            5   Pseudo R-squ. (CS):             0.3470
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5385      0.064     23.853      0.000       1.412       1.665
treatment[T.pbs]     0.2064      0.131      1.580      0.114      -0.050       0.462
====================================================================================
"""

In [30]:
# 8e-NRCH vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != 'max-SpRY')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -16.656
Date:                       Mon, 16 Jan 2023   Deviance:                       3.2317
Time:                               17:07:20   Pearson chi2:                     3.15
No. Iterations:                            5   Pseudo R-squ. (CS):             0.3490
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5013      0.101     14.810      0.000       1.303       1.700
treatment[T.pbs]     0.2436      0.152      1.600      0.110      -0.055       0.542
====================================================================================
"""

In [31]:
# stranded with tiers

In [32]:
strand0 = strand[strand['tier'] == 0]
bin_data_large = edit_df(strand0, col = 'AG')

In [33]:
# 8e-NRCH vs PBS r636q
data=bin_data_large[bin_data_large['mutation'] == 'r636q']
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -14.993
Date:                       Mon, 16 Jan 2023   Deviance:                      0.62441
Time:                               17:07:21   Pearson chi2:                    0.626
No. Iterations:                            5   Pseudo R-squ. (CS):             0.2542
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.3175      0.108     12.216      0.000       1.106       1.529
treatment[T.pbs]     0.2230      0.169      1.321      0.187      -0.108       0.554
====================================================================================
"""

In [34]:
# max-SpRY vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != '8e-NRCH')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -13.560
Date:                       Mon, 16 Jan 2023   Deviance:                       3.3619
Time:                               17:07:22   Pearson chi2:                     3.02
No. Iterations:                            5   Pseudo R-squ. (CS):             0.8089
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.3701      0.107     12.773      0.000       1.160       1.580
treatment[T.pbs]     0.7559      0.255      2.969      0.003       0.257       1.255
====================================================================================
"""

In [35]:
# 8e-NRCH vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != 'max-SpRY')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -12.294
Date:                       Mon, 16 Jan 2023   Deviance:                       2.8032
Time:                               17:07:23   Pearson chi2:                     2.47
No. Iterations:                            5   Pseudo R-squ. (CS):             0.6603
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.4098      0.172      8.191      0.000       1.072       1.747
treatment[T.pbs]     0.7161      0.288      2.487      0.013       0.152       1.281
====================================================================================
"""

In [36]:
strand1 = strand[strand['tier'] == 1]
bin_data_large = edit_df(strand1, col = 'AG')

In [37]:
# 8e-NRCH vs PBS r636q
data=bin_data_large[bin_data_large['mutation'] == 'r636q']
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -16.202
Date:                       Mon, 16 Jan 2023   Deviance:                       3.5703
Time:                               17:07:24   Pearson chi2:                     3.58
No. Iterations:                            5   Pseudo R-squ. (CS):             0.1813
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5913      0.116     13.683      0.000       1.363       1.819
treatment[T.pbs]     0.1908      0.175      1.093      0.274      -0.151       0.533
====================================================================================
"""

In [38]:
# max-SpRY vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != '8e-NRCH')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -14.598
Date:                       Mon, 16 Jan 2023   Deviance:                       2.7815
Time:                               17:07:25   Pearson chi2:                     3.10
No. Iterations:                            5   Pseudo R-squ. (CS):             0.1630
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.6644      0.118     14.071      0.000       1.433       1.896
treatment[T.pbs]    -0.2189      0.210     -1.042      0.297      -0.631       0.193
====================================================================================
"""

In [39]:
# 8e-NRCH vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != 'max-SpRY')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -14.188
Date:                       Mon, 16 Jan 2023   Deviance:                       4.2078
Time:                               17:07:25   Pearson chi2:                     4.23
No. Iterations:                            4   Pseudo R-squ. (CS):            0.01771
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5261      0.174      8.748      0.000       1.184       1.868
treatment[T.pbs]    -0.0806      0.246     -0.327      0.743      -0.563       0.402
====================================================================================
"""

In [40]:
strand2 = strand[strand['tier'] == 2]
bin_data_large = edit_df(strand2, col = 'AG')

In [41]:
# 8e-NRCH vs PBS r636q
data=bin_data_large[bin_data_large['mutation'] == 'r636q']
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -16.976
Date:                       Mon, 16 Jan 2023   Deviance:                       4.4541
Time:                               17:07:28   Pearson chi2:                     4.52
No. Iterations:                            5   Pseudo R-squ. (CS):             0.2144
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.3676      0.108     12.688      0.000       1.156       1.579
treatment[T.pbs]     0.2005      0.167      1.199      0.231      -0.127       0.528
====================================================================================
"""

In [42]:
# max-SpRY vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != '8e-NRCH')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -18.051
Date:                       Mon, 16 Jan 2023   Deviance:                       12.042
Time:                               17:07:30   Pearson chi2:                     10.8
No. Iterations:                            5   Pseudo R-squ. (CS):             0.1011
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5930      0.111     14.376      0.000       1.376       1.810
treatment[T.pbs]     0.1814      0.229      0.791      0.429      -0.268       0.631
====================================================================================
"""

In [43]:
# 8e-NRCH vs PBS p635l
data=bin_data_large[(bin_data_large['mutation'] == 'p635l') & (bin_data_large['treatment'] != 'max-SpRY')]
smf.glm("valuenonAG + valueAG ~ treatment ", family=sm.families.Binomial(), data=data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     ['valuenonAG', 'valueAG']   No. Observations:                    6
Model:                                   GLM   Df Residuals:                        4
Model Family:                       Binomial   Df Model:                            1
Link Function:                         Logit   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -13.349
Date:                       Mon, 16 Jan 2023   Deviance:                       3.3721
Time:                               17:07:31   Pearson chi2:                     3.43
No. Iterations:                            5   Pseudo R-squ. (CS):            0.09066
Covariance Type:                   nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5709      0.181      8.694      0.000       1.217       1.925
treatment[T.pbs]     0.2035      0.270      0.753      0.451      -0.326       0.733
====================================================================================
"""